# Workshop zu Image Segmentation am Beispiel U-Net

Autor: Joerg Dahlkemper
Version: 2022-06-15

Idee: https://pyimagesearch.com/2022/02/21/u-net-image-segmentation-in-keras/ und https://keras.io/examples/vision/oxford_pets_image_segmentation/

Literatur:
- U-Net Basisartikel https://arxiv.org/abs/1505.04597 
- DeepLab V3 https://arxiv.org/abs/1706.05587 
- HRNet https://arxiv.org/abs/1908.07919 
- U2-Net: https://arxiv.org/abs/2005.09007 

## Vorbereitung

Neben Tensorflow werden insbesondere die Bibliotheken labelme und albumentations benötigt, diese sind per pip einzubinden:

```pip install albumentations```<br>
```pip install labelme```

- Testen Sie anschließend die Ausführung der nachfolgenden imports und installieren Sie ggfs. fehlende Bibliotheken per ```pip install``` nach.
- Gehen Sie in Ihr lokals Verzeichnis des git-Repos ```aibv``` und führen Sie ```git pull``` aus, um die aktuellen Workshopdateien zu laden.

In [ ]:
import json
import os
import random

import albumentations as A
import cv2
import labelme
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img

## Daten aufbereiten: Labeln

- üblicherweise werden Bilddaten manuell per ```labelme``` annotiert, siehe https://github.com/wkentaro/labelme und dortige Verweise auf Tutorials (siehe dazu auch https://olafenwaayoola.medium.com/image-annotation-with-labelme-81687ac2d077 )
- in den damit erzeugten JSON-Dateien werden sowohl die Originalbilder als auch die Masken gespeichert

Labelme ist eine Python-Bibliothek, die in ein Python-Script eingebunden werden kann und Hilfsfunktionen, wie die Extraktion der Bildinformationen aus der JSON-Datei bietet. Das Programm zur Annotation von Bilddaten kann aber auch direkt aufgerufen werden, indem in der Eingabeaufforderung auf das gewünschte Bildverzeichnis gewechselt wird und dort Labelme als Python-Modul ausgeführt wird:

```python -m labelme```

**Aufgabe 1: Labeln eines Bildes**<br>
Aktualisieren Sie das Git-Repo der Vorlesung, indem Sie in das Verzeichnis ```aibv``` des git-Repos auf Ihrem Rechner wechseln und ```git pull``` ausführen. Labeln Sie eines der Bilder in dem Ordner ```aibv/workshop/tobelabeld``` entsprechend Ihrer Platznummer. Verwenden Sie dazu je nach Solarzellen einen oder mehrere der Label *crack*, *darkarea*, *finger*. Speichern Sie das Ergebnis und analysieren Sie die resultierende JSON-Datei mit VS Code oder einem anderen Texteditor. 

## Daten aufbereiten: Masken generieren

Zur Extraktion des Originalbildes und der Masken aus der JSON-Datei kann entweder auf fertige Skripte im Internet zurückgegriffen werden. Das Grundprinzip zur Erzeugung einer Maske kann aber auch sehr einfach durch Extraktion der in der JSON-Datei gespeicherten Punkte und Zeichnen eines entsprechenden Polygonzugs erfolgen, wie nachfolgend mit OpenCV und der Fuktion polyfill dargestellt wird.

**Aufgabe 2: Maske erzeugen**<br>
Analysieren Sie die Funktionsweise des folgenden Skriptbeispiels und testen Sie dies an dem von Ihnen gelabelten Bild. Erstellen Sie dazu ein Ablaufdiagramm für das Zeichnen der Labels und vergleichen Sie dies mit der Datenstruktur Ihres gelabelten JSON-Files.

In [ ]:
JSON_FILE = "tobelabeled/0009.json"


def label_scan(filename):

    labels = set()  # data structure with unique elements to avoid duplicates

    # read image and create a labelmap with background
    label_file = labelme.LabelFile(filename=JSON_FILE)
    img = labelme.utils.img_data_to_arr(label_file.imageData)  # noqa: F841

    with open(JSON_FILE) as json_file:
        json_label = json.loads(json_file.read())

        # walk trough all classes and generate label list
        for classes in json_label["shapes"]:
            labels.add(classes["label"])

    label_list = sorted(labels)
    print("[i] Labels found:", label_list)
    return label_list


def main():

    labels = label_scan(JSON_FILE)

    # read image and create a labelmap with background
    label_file = labelme.LabelFile(filename=JSON_FILE)
    img = labelme.utils.img_data_to_arr(label_file.imageData)
    img_file_name = JSON_FILE.split(".")[0] + "_from_json.jpg"
    mask_file_name = JSON_FILE.split(".")[0] + ".png"

    with open(JSON_FILE) as json_file:
        json_label = json.loads(json_file.read())
        img_height, img_width = img.shape
        label_map = np.zeros((img_height, img_width), np.uint8)

        for classes in json_label["shapes"]:

            # fill contour

            for i, label in enumerate(labels):
                if classes["label"] == label:
                    points = np.asarray(classes["points"], dtype=int)
                    cv2.fillPoly(label_map, pts=[points], color=i + 1)
                    print("[i] added segment", label)

        cv2.imwrite(img_file_name, img)
        cv2.imwrite(mask_file_name, label_map)

        plt.suptitle(JSON_FILE)
        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.title("Image")
        plt.subplot(1, 2, 2)
        plt.imshow(label_map)
        plt.title("Mask")
        plt.show()


if __name__ == "__main__":
    main()

## Daten einlesen

Es wird nachfolgend auf bereits gelabelte Bilder zugegriffen, die im git-Repo im Ordner segmentation abgelegt sind. In diesem Ordner befindet sich ein Ordner *image* für die Originalbilder und ein Ordner *masks* für die zugehörigen Masken.

**Aufgabe 3: Daten einlesen**<br>
Analysieren Sie die Codezeile zur Erstellung einer Liste der Bildpfade und lesen Sie in gleicher Weise die Liste der Masken ein. Machen Sie sich klar, dass die Sortierung später unbedingt erforderlich ist, da über den Listenindex auf das Bild und die zugehörige Maske zugegriffen wird.

Die Bild | Masken - Paare werden anschließend ausgegeben. Machen Sie sich klar, was die Funktion ```zip()``` in diesem typischen Anwendungsfall bewirkt.

In [ ]:
IMAGE_DIR = "segmentation/images"
MASK_DIR = "segmentation/masks"

image_paths = sorted(
    os.path.join(IMAGE_DIR, file_name)
    for file_name in os.listdir(IMAGE_DIR)
    if file_name.endswith(".jpg")
)

# TODO: Komplettieren Sie den fehlenden Code zur Erzeugung der Liste der Masken-Dateien
mask_paths = "TODO"

print("[i] Number of samples:", len(image_paths))

for image_path, mask_path in zip(image_paths, mask_paths):
    print(image_path, "|", mask_path)

**Aufgabe 4: Bildpaar anzeigen**<br>

- Lesen Sie dazu das Bild und die Maske über die Keras-Fumktion ```load_img``` ein und nutzen Sie bei der Maske den Modus ```load_img(..., color_mode='grayscale')```.
- Hinweis: Wenn Sie das Bild mit ```plt.imread``` einlesen, wird der ursprüngliche Wertebereich 0 ... 2 der Labels auf ein 255-stel verändert.
- Analysieren Sie die Form der Bilddaten und der Maske (wieviel Kanäle hat das Bild, welche Wertebereiche)
- Zeigen Sie unter Nutzung von matplotlib eines der Paare von Bild und Maske an. 
- Prüfen Sie, ob die Maske und das Bild zusammenpassen.
- Nutzen Sie bei der Anzeige der Maske, die Möglichkeit, den Farbbereich auf die Werte der Labels per ```imshow(..., vmin=0, vmax=2)``` einzugrenzen.

In [ ]:
# TODO: Anzeige eines bestimmten Bildpaares, z.B. 42 und überprüfen, ob Maske und Bild zusammenpassen

## Bild-Datengenerator ImageGenerator

Die Daten sind in keras-geeignete Datenstruktur zu überführen und Data augmentation ist erforderlich. Der imageDataGenerator von Keras erlaubt aber keine simultane Data Augmentation auf Bild und Maske. Für eine solche simultane Data Augmentation wird häufig die Bibliothek *albumentations* eingesetzt. Nachfolgend wird ein eigener Bilddaten-Generator implementiert.

**Aufgabe 5: Albumentations**<br>
Analysiseren Sie, wie das gleichzeitige Ändern von Bild und Maske implementiert wird, indem Sie die Dokumentationsseite von Albumentations aufrufen:
https://albumentations.ai/docs/getting_started/mask_augmentation/ 
Analysieren Sie die Funktion des nachfolgenden ImageGenerators und machen Sie sich klar, welche Art von Data Augmentation ausgeführt wird und ergänzen Sie die simultane Änderung von Bilddaten und zugehöriger Maske.

In [ ]:
class ImageGenerator(keras.utils.Sequence):
    """Helper allowing to iterate over the image data via for loop or next()"""

    def __init__(self, batch_size, img_size, image_paths, mask_paths, augment=True):
        self.batch_size = batch_size
        self.img_size = img_size
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.augment = augment  # flag for activation of the following augmentations
        self.transform = A.Compose(
            [
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.2),
            ]
        )

    def __len__(self):
        """returns the number of steps to cover the full data set"""
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, idx):
        """returns tuple (input, mask) corresponing to batch index"""
        i = idx * self.batch_size
        batch_image_paths = self.image_paths[i : i + self.batch_size]
        batch_mask_paths = self.mask_paths[i : i + self.batch_size]

        # create empty np-arrays for image (x) and label mask (y)
        x = np.zeros(
            (self.batch_size,) + self.img_size + (3,), dtype="float32"
        )  # concatenation of tupels to fit shape requirements
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")

        for j, (img_path, mask_path) in enumerate(
            zip(batch_image_paths, batch_mask_paths)
        ):
            img = (
                np.array(load_img(img_path, target_size=self.img_size), dtype="float32")
                / 255
            )
            mask = np.array(
                load_img(mask_path, target_size=self.img_size, color_mode="grayscale"),
                dtype="uint8",
            )
            mask = np.expand_dims(
                mask, 2
            )  # additional dimension for class number to fit required shape

            if self.augment:
                # TODO: Implementieren Sie hier die simultane Data augmentation gemäß der Anleitung von Albumentations

                img = "TODO"  # noqa: F841
                mask = "TODO"

            x[j] = img
            y[j] = mask  # additional dimension for class number to fit required shape

        return x, y

## Zugriff auf ImageGenerator

Der ImageGenerator erbt von der Klasse ```Sequence``` und implementiert die Methode ```__getitem__```, so dass die Batches per for-Schleife abgerufen werden können.

**Aufgabe 6: Anzeige eines Batches**<br>
Erstellen Sie einen ImageGenerator für eine Batch size von 3 und zeigen Sie die drei Bildpaare unter Nutzung von ```plt.subplot``` übersichtlich an. Dazu bietet es sich an, die For-Schleife nach einem Durchlauf per ```break``` zu beenden. Bei der Anzeige der Maske ist es notwendig, den Farbbereich auf dem Wertebereich der Labels, also 0 ... 2 per Option ```imshow(... ,vmin=0, vmax=2)``` einzuschränken.

In [ ]:
# TODO: ImageGenerator erzeugen und drei Bildpaare anzeigen

## Aufteilung von Trainings- und Validierungsdaten und Testdaten

Eine übliche Aufteilung wäre 70 % Trainingsbilder, 15% Validierungsbilder, 15% Testbilder (nur einmalig ganz am Projektende zu verwenden). Da hier nur eine sehr geringe Anzahl von 72 = 6 x 12 Bildern vorliegt, wird genau ein batch als Validierungsdatensatz genutzt, der Rest als Trainingsdaten. Es werden keine Testdaten vorgehalten.


**Aufgabe 7: Codeanalyse Data splitting**<br>
- Erklären Sie, wieso die Konstante SEED erforderlich ist.
- Visualisieren Sie als Skizze, wie die Aufteilung von Trainings- und Validierungsdaten per Doppelpunkt-Operator gelöst ist.

In [ ]:
BATCH_SIZE = 12  # depends on available memory
HEIGHT = 128
WIDTH = 128
SEED = 42

val_samples = 12

# Bilder und Masken zufällig anordnen
random.Random(SEED).shuffle(image_paths)
random.Random(SEED).shuffle(mask_paths)

train_image_paths = image_paths[:-val_samples]
train_mask_paths = mask_paths[:-val_samples]
val_image_paths = image_paths[-val_samples:]
val_mask_paths = mask_paths[-val_samples:]

# instantiate training and validation generators
train_gen = ImageGenerator(
    BATCH_SIZE, (HEIGHT, WIDTH), train_image_paths, train_mask_paths, True
)
val_gen = ImageGenerator(
    BATCH_SIZE, (HEIGHT, WIDTH), val_image_paths, val_mask_paths, False
)

## Hilfsprogramm zur Visualisierung der Daten

Je nachdem, ob nur ein Bildpaar (Original, Maske) oder ein Bildtriple (Original, Maske, Prediction) angezeigt werden soll, soll nachfolgende Funktion eine Liste von Originalbild, Maske und ggfs. dem Ergebnis der Segmentierung zur Anzeige bringen. Die Liste kann 2 oder 3 Bilder enthalten.

**Aufgabe 8: Analyse des Imshow-Befehls**<br>
Analysieren Sie, wozu die Methode ```squeeze()``` dient. Testen Sie dazu den nachfolgenden Code auch ohne diesen Methodenaufruf und interpretieren Sie die Fehlermeldung von ```imshow```.

In [ ]:
def display(display_list):
    plt.figure(figsize=(10, 10))
    title = ["Input Image", "True Mask", "Predicted Mask"]

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow((display_list[i]).squeeze(), vmin=0, vmax=2)
        plt.axis("off")
    plt.show()


sample_batch = next(
    iter(train_gen)
)  # train_batches does not support next, thus to be converted into iterable
random_index = np.random.choice(sample_batch[0].shape[0])
sample_image, sample_mask = sample_batch[0][random_index], sample_batch[1][random_index]
display([sample_image, sample_mask])

## Modell erstellen

Das Modell besteht aus dem U-förmigen Encoder und Decoder (auch Bottleneck genannt). Hierzu werden Conv-Layer mit der Aktivierungsfunktion ReLU genutzt. Es werden stets zwei Conv-Layer gleicher Filtergröße hintereinandergeschaltet, dazu bietet sich die Erstellung einer Funktion an. Darüber hinaus werden Blöcke für den Encoder zum Downsampling und Blöcke für den Decoder mit Upsampling benötigt, für die ebenfalls eigenständige Funktionen erstellt werden.

Da zur Erstellung der Skip Connections eine Concatenation erforderlich ist, bietet sich die functional API von Keras an.

**Aufgabe 9: U-Net Architektur ergänzen**<br>
Machen Sie sich anhand der Vorlesungsunterlagen den Aufbau des U-Nets klar und vervollständigen Sie den Encoder-Teil des nachfolgenden Codes um die fehlenden Layer. Die korrekte Umsetzung können Sie durch die übernächste Code-Zeile prüfen.

In [ ]:
def double_conv_block(x, n_filters):
    """generate sequence of 2 conv layers with same filter size"""
    x = layers.Conv2D(
        n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal"
    )(x)
    x = layers.Conv2D(
        n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal"
    )(x)
    return x


def downsample_block(x, n_filters):
    """generate downsample block for encoder"""
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p


def upsample_block(x, conv_features, n_filters):
    """generate upsample block for decoder"""
    x = layers.Conv2DTranspose(n_filters, kernel_size=3, strides=2, padding="same")(
        x
    )  # upsampling
    x = layers.concatenate([x, conv_features])  # concatenation for skip connection
    x = layers.Dropout(0.3)(x)
    x = double_conv_block(x, n_filters)
    return x


def build_unet_model(num_classes):
    """generation of unet model with decoder, bottleneck and encoder"""
    inputs = layers.Input(shape=(128, 128, 3))

    # encoder with downsampling
    f1, p1 = downsample_block(inputs, 64)
    f2, p2 = downsample_block(p1, 128)
    f3, p3 = downsample_block(p2, 256)
    f4, p4 = downsample_block(p3, 512)

    # bottleneck
    bottleneck = double_conv_block(p4, 1024)

    # decoder with upsampling
    u6 = upsample_block(bottleneck, f4, 512)  # noqa: F841
    # TODO: vervollstaendigen Sie die fehlenden Layer des U-Nets
    u7 = "TODO"  # noqa: F841
    u8 = "TODO"  # noqa: F841
    u9 = "TODO"  # noqa: F841

    # outputs
    outputs = layers.Conv2D(num_classes, 3, padding="same", activation="softmax")(u9)

    # functional definition of model
    unet_model = keras.Model(inputs, outputs, name="U-Net")

    return unet_model

Bei erfolgreicher Umsetzung lässt sich das Modell hiermit bauen und es ergibt bei dem Aufruf von ```unet_model.summary()``` sich die Anzahl von 34.515.011 zu trainierenden Parametern.

In [ ]:
unet_model = build_unet_model(3)
unet_model.summary()

## Training

Für die semantische Segmentierung bei sich ausschließenden Klassen (jedes Pixel kann nur einer Klasse zugeordnet werden), wird in der Regel die Kostenfunktion ```sparse_categorical_crossentropy``` verwendet.

Je nach Leistungsfähigkeit des Rechners variiert die Ausführungszeit je Epoche von etwa 70 ms bei Einsatz einer Graphikkarte bis zu 10 s. Entsprechend ist nachfolgendes Training zunächst mit einer geringen Anzahl von Epochen zu starten. Erste brauchbare Ergebnisse sind bereits bei 50 Epochen zu erwarten. Wünschenswert ist ein Training über 200 Epochen.

In [ ]:
NUM_EPOCHS = 5  # ab 50 Epochen brauchbare Werte, 200 Epochen erstrebenswert, entsprechend anpassen

unet_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics="accuracy",
)
model_history = unet_model.fit(train_gen, epochs=NUM_EPOCHS, validation_data=val_gen)
unet_model.save("my_model.h5")

## Visualisierung des Trainingsverlaufs

Nachfolgend wird der Trainingsverlauf dargestellt.

**Aufgabe 10: Trainingsverlauf analysieren**<br>
Analysieren Sie den unterschiedlichen Verlauf von Loss und Accuracy und überlegen Sie, was an dem Verlauf der Accuracy ungewöhnlich ist und welche Ursache dies haben kann.

In [ ]:
loss = model_history.history["loss"]
val_loss = model_history.history["val_loss"]
acc = model_history.history["accuracy"]
val_acc = model_history.history["val_accuracy"]

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.plot(loss, label="loss")
plt.plot(val_loss, label="val_loss")
plt.title("Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(acc, label="acc")
plt.plot(val_acc, label="val_acc")
plt.title("Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.grid()
plt.show()

## Prediction

Sofern das Modell bereits erstellt und abgespeichert wurde kann es durch Ausführen der folgenden Zelle eingelesen werden, ohne das Training zu wiederholen. Wenn das eigene Training zu langsam ist, können Sie auf das im Git-Repo hinterlegte Modell ``` master_model.h5``` zugreifen.

In [ ]:
# unet_model = keras.models.load_model("my_model.h5")
# unet_model = keras.models.load_model("master_model.h5")

## Visualisierung der Ergebnisse

Um einen qualitativen Eindruck der Leistungsfähigkeit der Lösung zu erhalten, sollen die Ergebnisse der Vorhersage auf dem Validierungsdatensatz visualisiert werden. Hierzu wird nachfolgend eine Prediction auf dem Validierungsdatensatz ausgeführt.

**Aufgabe 11: Analyse des Ausgabeformats**<br>
Werten Sie jeweils von der Maske und der vorhergesagten Maske die Anzahl der Dimensionen und deren Wertebereiche aus und erklären Sie den Unterschied.


In [ ]:
images, masks = next(
    iter(val_gen)
)  # convert val_gen in iterable to get single batch by next
predicted_masks = unet_model.predict(images)

In [ ]:
# TODO: Analysieren Sie die shape der Tensoren images, masks und predicted_masks und den Wertebereich von predicted_masks

**Aufgabe 12: Umwandlung der Vorhersage**<br>
Analysieren Sie, wie die Umwandlung der vorhergesagten Maske in der Funktion ```multi2one_channel_mask``` funktioniert und führen Sie dann die nachfolgende Zelle aus.

In [ ]:
def multi2one_channel_mask(multi_channel_mask):
    """
    image returns one channel per class with probability values, to display
    the object and outline, just the channel with the highest value is selected
    """
    one_channel_mask = np.argmax(multi_channel_mask, axis=-1)
    output_mask = np.expand_dims(one_channel_mask, axis=-1)
    return output_mask


for i in range(len(images)):
    display(
        [images[i], masks[i], multi2one_channel_mask(predicted_masks[i])]
    )  # 3 channel output converted to 1 channel with 3 values

## Auswertung

**Aufgabe 13: Bewertung der Ergebnisse**<br>

1. Bewerten Sie die Ergebnisse der semantischen Segmentierung qualitativ und vergleichen Sie diese vor dem Hintergrund der Segmentierung mittels klassischer Bildverarbeitung.
2. Machen Sie sich klar, welche Operationen der klassischen Bildverarbeitung im Anschluss an die Segmentierung erforderlich sind, um eine finale Auswahl von fehlerhaften Solarzellen und deren Klassifizierung zu ermöglichen.